# Install conda

In [1]:
import sys
! wget https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh
! chmod +x Anaconda3-2020.02-Linux-x86_64.sh
! bash ./Anaconda3-2020.02-Linux-x86_64.sh -b -f -p /usr/local
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda update -n base -c defaults conda -y
! conda config --add channels bioconda
! conda config --add channels conda-forge
! conda install biopython -y

--2020-07-25 02:52:48--  https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 546910666 (522M) [application/x-sh]
Saving to: ‘Anaconda3-2020.02-Linux-x86_64.sh’

Anaconda3-2020.02-L 100%[===================>] 521.57M   201MB/s    in 2.6s    

2020-07-25 02:52:51 (201 MB/s) - ‘Anaconda3-2020.02-Linux-x86_64.sh’ saved [546910666/546910666]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - anaconda-navigator==1.9.12=py37_0
    - anacon

# Create and set up virtual environment for antiSMASH

In [2]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell

# Create virtual environment for antiSMASH, then install dependencies, antiSMASH, and databases into it (this will take a while)
conda create --prefix /usr/local/envs/antismash antismash -y
conda activate antismash

# Download antiSMASH databases and check that everything was installed properly
download-antismash-databases
antismash --check-prereqs
conda deactivate

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local/envs/antismash

  added / updated specs:
    - antismash


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            0_gnu         435 KB  conda-forge
    antismash-5.1.2            |             py_2        35.9 MB  bioconda
    bcbio-gff-0.6.6            |             py_0          17 KB  bioconda
    blast-2.5.0                |       hc0b0e79_3       137.8 MB  bioconda
    boost-1.70.0               |   py37h9de70de_1         337 KB  conda-forge
    boost-cpp-1.70.0           |       h7b93d67_3        21.1 MB  conda-forge
    bx-python-0.8.9            |   py37h029c07a_1         1.0 

# Download sample genome to test antiSMASH

In [7]:
!pip install biopython
from Bio import Entrez

accession = 'NC_013216'
output_path="/usr/local/envs/antismash"
output_string='%s/%s.gbk'%(output_path, accession.strip())

handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gbwithparts", retmode="text")
print('Download of %s successful'%(accession.strip()))
print('Reading ' + output_string + '...')
record_string=handle.read()
with open(output_string, 'w') as record_writer:
    record_writer.write(record_string)
print('Saved as: ' + output_string)

/usr/local/lib/python3.7/site-packages/Bio/Entrez/__init__.py:668: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  UserWarning,


Download of NC_013216 successful
Reading /usr/local/envs/antismash/NC_013216.gbk...
Saved as: /usr/local/envs/antismash/NC_013216.gbk


# Run antiSMASH on a single example genome

In [8]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell

conda activate antismash

echo "Processing with antiSMASH..."
antismash /usr/local/envs/antismash/NC_013216.gbk \
  --output-dir /usr/local/envs/antismash/NC_013216

conda deactivate

ls /usr/local/envs/antismash/NC_013216

Processing with antiSMASH...
/usr/local/envs/antismash/lib/python3.7/site-packages/scss/selector.py:54: FutureWarning: Possible nested set at position 329
  ''', re.VERBOSE | re.MULTILINE)
css	    NC_013216.1.region001.gbk  NC_013216.1.region005.gbk  NC_013216.zip
images	    NC_013216.1.region002.gbk  NC_013216.1.region006.gbk  regions.js
index.html  NC_013216.1.region003.gbk  NC_013216.gbk		  svg
js	    NC_013216.1.region004.gbk  NC_013216.json


# Run antiSMASH on several genomes

In [ ]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell

set -o errexit
set -o pipefail

conda activate antismash

# Specify input and output directories
input_dir="<Path to directory containg .gbk files>" # No slash at the end of path
output_dir="<Path to desired output directory>" # No slash at the end of path

# Make output directory if it doesn't exist
mkdir -p "$output_dir"

# Run antiSMASH on files in input_dir
for file in "$input_dir"/*.gbk
do
  filename=$(basename "$file" ".gbk") # Extract refseq ID from filename
  output_subdir="$output_dir/$filename" # Define output subdirectory based on refseq ID
  if [ ! -d "$output_subdir" ]
  then # run if output_subdir doesn't exist
    echo $file
    echo "Processing with antiSMASH"
    # Run antiSMASH on file and save output in output subdirectory
    antismash "$file" \
      --output-dir "$output_subdir" \
      --taxon bacteria \
      --genefinding-tool prodigal \
      --pfam2go \
      --clusterhmmer 
  else
    echo "$file already processed with antiSMASH"
  fi
done

conda deactivate